In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
# Leitura e extração dos dados pelo método CSV, onde durante a leitura os dados ausentes serão substuídos por dados não informados/dados não disponíveis

valores_ausentes = ['**','###!','####','****','*****','NULL']
df = pd.read_csv('ocorrencia.csv', sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

In [ ]:
# Definindo os tipos das colunas e indicando quais não possuem valores NA

schema = pa.DataFrameSchema(
    columns= {
        "codigo": pa.Column(pa.Int, required=False),
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [ ]:
# Verificação da alteração de tipo de dado de cada coluna

df.dtypes

In [ ]:
# Retornando lable da segunda linha

df.loc[1]

Manipulação de Valores Nulos

A função _isna_ trata valores não informados, mesmo com outra característica, outro valor, como o NaN, ou seja, indepentente se nesse df possui NA, NaT, NaN, o _Pandas_ entende esses valores como valores não disponíveis/não informados

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum()

In [ ]:
# Valores Nulos por Estado

df.loc[df.ocorrencia_uf.isnull()]

In [ ]:
filtro = df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

In [ ]:
filtro = df.ocorrencia_uf.isnull()
df.loc[filtro]

In [ ]:
#Pelo count sabemos a quantidade de ocorrências com valores nulos porque ele por padrão ele não conta os valores nulos, é só vermos a diferença dos valores, já o filtro sabemos quais são as ocorrências nulas

In [ ]:
filtro = df.ocorrencia_hora.isnull()
df.loc[filtro]

In [ ]:
# Ocorrências com mais de 10 recomendações | Todas as colunas

filtro = df.total_recomendacoes > 10
df.loc[filtro]

In [ ]:
# Cidades que tiveram ocorrências com mais de 10 recomendações
 
df.loc[filtro, 'ocorrencia_cidade']

In [ ]:
# Cidades que tiveram ocorrências com mais de 10 recomendações e o total 
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade','total_recomendacoes']]

In [ ]:
# Filtrando as ocorrências que tiveram incidentes graves

filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

In [ ]:
# Ocorrências de incidentes graves em São Paulo

filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

In [ ]:
# Ocorrências cuja cidade comece com a Letra C

filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

In [ ]:
# Ocorrências cuja cidade termine com a Letra A

filtro = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro]

In [ ]:
# Ocorrências cuja cidade termine com os caracteres MA

filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro]

In [ ]:
# Ocorrências cuja cidades contenha em seu nome os caracteres MA

filtro = df.ocorrencia_cidade.str.contains ('MA')
df.loc[filtro]

In [ ]:
# Ocorrências do ano de 2015

filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

In [ ]:
# Ocorrências de Dezembro de 2015
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

In [ ]:
# Ocorrências de 08 de Dezembro de 2015

filtro_ano = df.ocorrencia_dia.dt.year == 2015
filtro_mes = df.ocorrencia_dia.dt.month == 12
filtro_dia = df.ocorrencia_dia.dt.day == 8
df.loc[filtro_ano & filtro_mes & filtro_dia]

In [ ]:
# Concatenando colunas Ocorrência dia e Ocorrência hora para consulta mais aprimorada

df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [ ]:
# Retornando ocorrências em com data e horas específicas

filtro_1 = df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro_2 = df.ocorrencia_dia_hora <= '2015-12-08 14:00:00'
df.loc[filtro_1 & filtro_2]

In [ ]:
# Agrupamento de Dados e Criação de Novo DF referente a março de 2015

filtro_1 = df.ocorrencia_dia.dt.year == 2015
filtro_2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro_1 & filtro_2]
df201503

In [ ]:
df201503.groupby(['codigo_ocorrencia']).count()

In [ ]:
# Contando dentro de Ocorrência quantos  ACIDENTEs, INCIDENTEs, INCINDENTEs GRAVEs ocorreram

df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

In [ ]:
# SIZE | Agrupa e conta os registros, que foram agrupados, as linha do Dataframe

df201503.groupby(['ocorrencia_classificacao']).size()

In [ ]:
# Contando número de linhas agrupadas do menor pro maior

df201503.groupby(['ocorrencia_classificacao']).size().sort_values()

In [ ]:
# Contando número de linhas agrupadas do maior pro menor

df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

In [ ]:
# Filtrando Ocorrências da Região Sudeste

filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
dfsudeste2015 = df.loc[filtro1 & filtro2]
dfsudeste2015

In [ ]:
# Retornadndo quantidade de incidentes na região sudeste

dfsudeste2015.count()

In [ ]:
# Retronando incidentes do ano de 2015

dfsudeste2015.groupby(['ocorrencia_classificacao']).size()

In [ ]:
# Retornando quantas ocorrências por classificação tiveram em cada estado do sudeste

dfsudeste2015.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()

In [ ]:
# Retornando os incidentes por cidade no sudeste de forma decrescente

dfsudeste2015.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

In [ ]:
# Retornando Ocorrências da cidade do Rio de Janeiro

filtro = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2015.loc[filtro]

In [ ]:
# Total de Recomendações que o Rio de Janeiro teve

filtro = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
dfsudeste2015.loc[filtro].total_recomendacoes.sum()

In [ ]:
# Confirmando as 6 recomendações que resultaram do código anterior, verificar na coluna total_recomendacoes

filtro1 = dfsudeste2015.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = dfsudeste2015.total_recomendacoes > 0
dfsudeste2015.loc[filtro1 & filtro2]

In [ ]:
# Total de Recomendações por cidade da Região Sudeste

dfsudeste2015.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

In [ ]:
# Total de Recomendações por Aeródromo

dfsudeste2015.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

In [ ]:
# Retornando somente as cidades que tem recomendações não nulas ordenadas por ordem crescente

filtro = dfsudeste2015.total_recomendacoes > 0
dfsudeste2015.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values

In [ ]:
# Retornando as ocorrências da cidade de Viçosa que possuem pele menos 1 ocorrência

filtro1 = dfsudeste2015.total_recomendacoes > 0 
filtro2 = dfsudeste2015.ocorrencia_cidade == 'VIÇOSA'
dfsudeste2015.loc[filtro1 & filtro2]